### Webtoon 회차별 정보
- 웹툰명 : title
- 회차 : no
- 웹툰 회차의 기본키(ID값)
- 회차별 썸네일링크

In [2]:
import requests
from bs4 import BeautifulSoup
import re

In [20]:
# 함수화
def naver_webtoon(genre):
    
    dic = {}
    dic['unique_id'] = []
    dic['title'] = []
    dic['author'] = []
    dic['total_score'] = []
    dic['main_link'] = []
    dic['image_link'] = []
    dic['update_date'] = []
    
    # first url for title
    # get title with full name
    url = "https://comic.naver.com/webtoon/genre.nhn?view=list&order=ViewCount&genre=" + genre
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select("#content > div.list_area.table_list_area > table > tbody > tr")
    for keyword in keywords:
        dic['title'].append(keyword.select_one("td > a > strong").text)
    
    
    # second url for the others
    url =  "https://comic.naver.com/webtoon/genre.nhn?genre=" + genre
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select(".list_area > .img_list > li")


    # author, total score
    for keyword in keywords:
        dic['author'].append(keyword.select_one("dl > dd > a").text)
        dic['total_score'].append(keyword.select_one("dl > dd > .rating_type > strong").text)
        dic['update_date'].append(keyword.select_one("dl > dd.date2").text)

    # unique id 
    unique_id_list = dom.select("dl > dt > a")
    for i in range(len(unique_id_list)):
        idx = unique_id_list[i].get('href').index("=")
        dic['unique_id'].append(unique_id_list[i].get('href')[idx+1:])

    # main link
    main_link_list = dom.select("dl > dt > a")
    for i in range(len(main_link_list)):
        dic['main_link'].append("https://comic.naver.com" + main_link_list[i].get('href'))

    # image link
    image_link_list = dom.select(".list_area > .img_list > li > .thumb > a > img:nth-of-type(1)")
    for i in range(len(image_link_list)):
        dic['image_link'].append(image_link_list[i].get('src'))
        
    df = pd.DataFrame(dic)
    df[genre] = genre
    
    return df

In [21]:
genres = ["episode","omnibus","story","daily","comic","fantasy","action","drama","pure","sensibility","thrill","historical","sports"]

In [22]:
df = naver_webtoon("episode")

In [23]:
for genre in genres[1:]:
    print(genre)
    sample = naver_webtoon(genre)
    df = pd.merge(df,sample, how='outer')
df.reset_index(inplace=True, drop=True)
df.tail()

omnibus
story
daily
comic
fantasy
action
drama
pure
sensibility
thrill
historical
sports


,unique_id,title,author,total_score,main_link,image_link,update_date,episode,omnibus,story,daily,comic,fantasy,action,drama,pure,sensibility,thrill,historical,sports
740,622643,진눈깨비 소년,쥬드프라이데이,9.97,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2018.08.30,NaN,NaN,story,NaN,NaN,NaN,NaN,NaN,pure,sensibility,NaN,NaN,NaN
741,703840,한국만화 1990,웹툰작가,9.75,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2018.05.17,NaN,NaN,story,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
742,336945,길에서 만나다,쥬드 프라이데이,9.75,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2013.01.17,NaN,NaN,story,NaN,NaN,NaN,NaN,drama,NaN,sensibility,NaN,NaN,NaN
743,515883,사또(Satto),최윤진,9.80,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2015.12.29,NaN,NaN,story,NaN,NaN,fantasy,NaN,drama,NaN,NaN,NaN,NaN,NaN
744,347685,슈퍼트리오 시즌2,황미나,7.36,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2012.06.04,NaN,NaN,story,NaN,comic,NaN,NaN,drama,NaN,NaN,NaN,NaN,NaN


In [24]:
def make_genre(df):
    genres = ["episode","omnibus","story","daily","comic","fantasy","action","drama","pure","sensibility","thrill","historical","sports"]
    df.fillna("", inplace=True)

    result = []
    for i in range(len(df[genres])):
        ls = list(filter(lambda value: value != "", df[genres].loc[i].values))
        result.append(ls)

    df['genre'] = np.array(["/".join(ls) for ls in result])
    df.drop(columns=genres, axis=1, inplace=True)
    return df

In [25]:
df = make_genre(df)
df.tail()

,unique_id,title,author,total_score,main_link,image_link,update_date,genre
740,622643,진눈깨비 소년,쥬드프라이데이,9.97,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2018.08.30,story/pure/sensibility
741,703840,한국만화 1990,웹툰작가,9.75,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2018.05.17,story
742,336945,길에서 만나다,쥬드 프라이데이,9.75,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2013.01.17,story/drama/sensibility
743,515883,사또(Satto),최윤진,9.80,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2015.12.29,story/fantasy/drama
744,347685,슈퍼트리오 시즌2,황미나,7.36,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2012.06.04,story/comic/drama


In [26]:
titleid = df.unique_id.values

In [38]:
ti

'유미의 세포들'

아래 코드에서 num range는 파악이 가능하다. 즉, 링크에서 href안에 'no=326' 과 같이 표기가 되어있어 이를 10으로 나누어 page수 파악이 가능하다.

In [85]:
dic = {}
dic['title'] = []
dic['image_link'] = []
dic['episode'] = []


count = 1
for tid in titleid:
#     url = "https://comic.naver.com/webtoon/list.nhn?titleId="+ str(tid) +"&page=1"
#     response = requests.get(url)
#     dom = BeautifulSoup(response.content, "html.parser")
#     num_check = dom.select("#content > table.viewList > tr > td:nth-of-type(1) > a > img")
    print("webtoon id:",tid, " count: ", count)
    for num in range(1,201):
        
        url = "https://comic.naver.com/webtoon/list.nhn?titleId="+ str(tid) +"&page=" + str(num)
        response = requests.get(url)
        dom = BeautifulSoup(response.content, "html.parser")
        
        # title
        dic['title'].append(df[df.unique_id == tid].title.values[0])
        
        # image link
        # 첫번째 링크가 광고인 경우를 제외시켜줘야 한다.
        links = dom.select("#content > table.viewList > tr > td:nth-of-type(1) > a > img")
        if len(links) > 10:
            links = links[1:]
        dic['image_link'].append(np.array([link.get('src') for link in links]))
        # 회차(episode)
        dic['episode'].append(np.array([link.get('title') for link in links]))
        if len(links) < 10:
            break 
    count += 1

webtoon id: 651673  count:  1
webtoon id: 22052  count:  2
webtoon id: 498158  count:  3
webtoon id: 680911  count:  4
webtoon id: 15441  count:  5
webtoon id: 26316  count:  6
webtoon id: 409630  count:  7
webtoon id: 710747  count:  8
webtoon id: 679519  count:  9
webtoon id: 15938  count:  10
webtoon id: 651663  count:  11
webtoon id: 25517  count:  12
webtoon id: 639604  count:  13
webtoon id: 113121  count:  14
webtoon id: 316914  count:  15
webtoon id: 26101  count:  16
webtoon id: 70046  count:  17
webtoon id: 563782  count:  18
webtoon id: 20853  count:  19
webtoon id: 710751  count:  20
webtoon id: 26144  count:  21
webtoon id: 55169  count:  22
webtoon id: 52946  count:  23
webtoon id: 651664  count:  24
webtoon id: 632342  count:  25
webtoon id: 55143  count:  26
webtoon id: 609477  count:  27
webtoon id: 50422  count:  28
webtoon id: 642598  count:  29
webtoon id: 92106  count:  30
webtoon id: 710754  count:  31
webtoon id: 650304  count:  32
webtoon id: 316912  count:  33


webtoon id: 511447  count:  263
webtoon id: 25695  count:  264
webtoon id: 517773  count:  265
webtoon id: 26460  count:  266
webtoon id: 557676  count:  267
webtoon id: 702422  count:  268
webtoon id: 597481  count:  269
webtoon id: 459546  count:  270
webtoon id: 677737  count:  271
webtoon id: 670149  count:  272
webtoon id: 710761  count:  273
webtoon id: 552960  count:  274
webtoon id: 710760  count:  275
webtoon id: 26108  count:  276
webtoon id: 26107  count:  277
webtoon id: 142911  count:  278
webtoon id: 670150  count:  279
webtoon id: 651617  count:  280
webtoon id: 570506  count:  281
webtoon id: 25050  count:  282
webtoon id: 713975  count:  283
webtoon id: 698918  count:  284
webtoon id: 655748  count:  285
webtoon id: 699658  count:  286
webtoon id: 478262  count:  287
webtoon id: 337964  count:  288
webtoon id: 258205  count:  289
webtoon id: 714839  count:  290
webtoon id: 537990  count:  291
webtoon id: 551651  count:  292
webtoon id: 511446  count:  293
webtoon id: 6

webtoon id: 300957  count:  521
webtoon id: 58293  count:  522
webtoon id: 632705  count:  523
webtoon id: 452117  count:  524
webtoon id: 639080  count:  525
webtoon id: 656641  count:  526
webtoon id: 679568  count:  527
webtoon id: 629056  count:  528
webtoon id: 710752  count:  529
webtoon id: 579352  count:  530
webtoon id: 602930  count:  531
webtoon id: 697681  count:  532
webtoon id: 316913  count:  533
webtoon id: 703853  count:  534
webtoon id: 58219  count:  535
webtoon id: 462900  count:  536
webtoon id: 622646  count:  537
webtoon id: 710763  count:  538
webtoon id: 449854  count:  539
webtoon id: 604146  count:  540
webtoon id: 613933  count:  541
webtoon id: 703849  count:  542
webtoon id: 665170  count:  543
webtoon id: 710741  count:  544
webtoon id: 546623  count:  545
webtoon id: 703634  count:  546
webtoon id: 657948  count:  547
webtoon id: 703631  count:  548
webtoon id: 644112  count:  549
webtoon id: 709731  count:  550
webtoon id: 459545  count:  551
webtoon id

In [169]:
url = "https://comic.naver.com/webtoon/list.nhn?titleId=651673&page=2"
response = requests.get(url)
dom = BeautifulSoup(response.content, "html.parser")
# episodes = dom.select("#content > table.viewList > tr:nth-of-type(2), tr:nth-of-type(3), tr:nth-of-type(4), tr:nth-of-type(5), tr:nth-of-type(6), tr:nth-of-type(7), tr:nth-of-type(8), tr:nth-of-type(9), tr:nth-of-type(10), tr:nth-of-type(11)")
episodes = dom.select("#content > table.viewList > tr > td:nth-of-type(1) > a")
print(len(episodes))
a = []
for i in range(len(episodes)):
    a.append(episodes[i].get('onclick'))
a

11


["callWebAdBannerClick('/api/stat/gameBanner.nhn?type=CLICK&target=ARTICLE_LIST&bannerId=128', 'https://yumiscells.superbcorp.com/');clickcr(this,'lst.adbanner','','',event);return false;",
 "clickcr(this,'lst.img','651673','326',event)",
 "clickcr(this,'lst.img','651673','325',event)",
 "clickcr(this,'lst.img','651673','324',event)",
 "clickcr(this,'lst.img','651673','323',event)",
 "clickcr(this,'lst.img','651673','322',event)",
 "clickcr(this,'lst.img','651673','321',event)",
 "clickcr(this,'lst.img','651673','320',event)",
 "clickcr(this,'lst.img','651673','319',event)",
 "clickcr(this,'lst.img','651673','318',event)",
 "clickcr(this,'lst.img','651673','317',event)"]

In [186]:
int(a[1].split(",")[3][1:-1])

326

In [109]:
episodes[0].get('src')

'https://naverwebtoon-phinf.pstatic.net/20180817_257/1534482725737dKSDa_JPEG/upload_8025313691756401672.JPEG'

In [99]:
first = dic['image_link'][0]
for array in dic['image_link'][1:]:
    first = np.hstack([first, array])
first

array(['https://shared-comic.pstatic.net/thumb/webtoon/651673/336/thumbnail_202x120_d30a7b42-abe2-43a7-b90b-ddfb54d71f16.jpg',
       'https://shared-comic.pstatic.net/thumb/webtoon/651673/335/thumbnail_202x120_09458cd2-25cf-47a0-94f2-37fac1c1f165.jpg',
       'https://shared-comic.pstatic.net/thumb/webtoon/651673/334/thumbnail_202x120_7ce2d3c2-d510-4129-91a2-be4866847db0.jpg',
       ...,
       'https://shared-comic.pstatic.net/thumb/webtoon/347685/4/inst_thumbnail_20110718150049.jpg',
       'https://shared-comic.pstatic.net/thumb/webtoon/347685/3/inst_thumbnail_20110711143941.jpg',
       'https://shared-comic.pstatic.net/thumb/webtoon/347685/2/inst_thumbnail_20110705155124.jpg'],
      dtype='<U117')

In [100]:
dic['image_link_combined'] = first

In [103]:
second = dic['episode'][0]
for array in dic['episode'][1:]:
    second = np.hstack([second, array])

In [93]:
for k in dic.keys():
    print(len(dic[k]))

19112
19112
19112


In [115]:
dic1 = {}
dic1['title'] = []
dic1['image_link'] = []
dic1['episode'] = []


count = 1
for tid in titleid:
#     url = "https://comic.naver.com/webtoon/list.nhn?titleId="+ str(tid) +"&page=1"
#     response = requests.get(url)
#     dom = BeautifulSoup(response.content, "html.parser")
#     num_check = dom.select("#content > table.viewList > tr > td:nth-of-type(1) > a > img")
    print("webtoon id:",tid, " count: ", count)
    for num in range(1,201):
        
        url = "https://comic.naver.com/webtoon/list.nhn?titleId="+ str(tid) +"&page=" + str(num)
        response = requests.get(url)
        dom = BeautifulSoup(response.content, "html.parser")
        
        # image link
        # 첫번째 링크가 광고인 경우를 제외시켜줘야 한다.
        links = dom.select("#content > table.viewList > tr > td:nth-of-type(1) > a > img")
        if len(links) > 10:
            links = links[1:]        
        # title
        for _ in range(len(links)):
            dic1['title'].append(df[df.unique_id == tid].title.values[0])
        

        dic1['image_link'].append(links[i].get('src') for i in range(len(links)))
        # 회차(episode)
        dic1['episode'].append(links[i].get('title') for i in range(len(links)))
        if len(links) < 10:
            break 
    count += 1

webtoon id: 651673  count:  1
webtoon id: 22052  count:  2
webtoon id: 498158  count:  3
webtoon id: 680911  count:  4
webtoon id: 15441  count:  5
webtoon id: 26316  count:  6
webtoon id: 409630  count:  7
webtoon id: 710747  count:  8
webtoon id: 679519  count:  9
webtoon id: 15938  count:  10
webtoon id: 651663  count:  11
webtoon id: 25517  count:  12
webtoon id: 639604  count:  13
webtoon id: 113121  count:  14
webtoon id: 316914  count:  15
webtoon id: 26101  count:  16
webtoon id: 70046  count:  17
webtoon id: 563782  count:  18
webtoon id: 20853  count:  19
webtoon id: 710751  count:  20
webtoon id: 26144  count:  21
webtoon id: 55169  count:  22
webtoon id: 52946  count:  23
webtoon id: 651664  count:  24
webtoon id: 632342  count:  25
webtoon id: 55143  count:  26
webtoon id: 609477  count:  27
webtoon id: 50422  count:  28
webtoon id: 642598  count:  29
webtoon id: 92106  count:  30
webtoon id: 710754  count:  31
webtoon id: 650304  count:  32
webtoon id: 316912  count:  33


webtoon id: 511447  count:  263
webtoon id: 25695  count:  264
webtoon id: 517773  count:  265
webtoon id: 26460  count:  266
webtoon id: 557676  count:  267
webtoon id: 702422  count:  268
webtoon id: 597481  count:  269
webtoon id: 459546  count:  270
webtoon id: 677737  count:  271
webtoon id: 670149  count:  272
webtoon id: 710761  count:  273
webtoon id: 552960  count:  274
webtoon id: 710760  count:  275
webtoon id: 26108  count:  276
webtoon id: 26107  count:  277
webtoon id: 142911  count:  278
webtoon id: 670150  count:  279
webtoon id: 651617  count:  280
webtoon id: 570506  count:  281
webtoon id: 25050  count:  282
webtoon id: 713975  count:  283
webtoon id: 698918  count:  284
webtoon id: 655748  count:  285
webtoon id: 699658  count:  286
webtoon id: 478262  count:  287
webtoon id: 337964  count:  288
webtoon id: 258205  count:  289
webtoon id: 714839  count:  290
webtoon id: 537990  count:  291
webtoon id: 551651  count:  292
webtoon id: 511446  count:  293
webtoon id: 6

webtoon id: 300957  count:  521
webtoon id: 58293  count:  522
webtoon id: 632705  count:  523
webtoon id: 452117  count:  524
webtoon id: 639080  count:  525
webtoon id: 656641  count:  526
webtoon id: 679568  count:  527
webtoon id: 629056  count:  528
webtoon id: 710752  count:  529
webtoon id: 579352  count:  530
webtoon id: 602930  count:  531
webtoon id: 697681  count:  532
webtoon id: 316913  count:  533
webtoon id: 703853  count:  534
webtoon id: 58219  count:  535
webtoon id: 462900  count:  536
webtoon id: 622646  count:  537
webtoon id: 710763  count:  538
webtoon id: 449854  count:  539
webtoon id: 604146  count:  540
webtoon id: 613933  count:  541
webtoon id: 703849  count:  542
webtoon id: 665170  count:  543
webtoon id: 710741  count:  544
webtoon id: 546623  count:  545
webtoon id: 703634  count:  546
webtoon id: 657948  count:  547
webtoon id: 703631  count:  548
webtoon id: 644112  count:  549
webtoon id: 709731  count:  550
webtoon id: 459545  count:  551
webtoon id

In [116]:
for k in dic1.keys():
    print(len(dic1[k]))

187691
19112
19112


In [119]:
dic1['episode']

[<generator object <genexpr> at 0x000001C702A06C50>,
 <generator object <genexpr> at 0x000001C70242F620>,
 <generator object <genexpr> at 0x000001C703D1ED00>,
 <generator object <genexpr> at 0x000001C702B250A0>,
 <generator object <genexpr> at 0x000001C702AD4468>,
 <generator object <genexpr> at 0x000001C7040C3360>,
 <generator object <genexpr> at 0x000001C702869780>,
 <generator object <genexpr> at 0x000001C70421D410>,
 <generator object <genexpr> at 0x000001C708BB5780>,
 <generator object <genexpr> at 0x000001C70449CF10>,
 <generator object <genexpr> at 0x000001C7058D9A40>,
 <generator object <genexpr> at 0x000001C7045024C0>,
 <generator object <genexpr> at 0x000001C706A4B0A0>,
 <generator object <genexpr> at 0x000001C7058D9678>,
 <generator object <genexpr> at 0x000001C70449C518>,
 <generator object <genexpr> at 0x000001C7026AD3B8>,
 <generator object <genexpr> at 0x000001C702C805C8>,
 <generator object <genexpr> at 0x000001C702117258>,
 <generator object <genexpr> at 0x000001C70119

In [121]:
dic2 = {}
dic2['title'] = []
dic2['image_link'] = []
dic2['episode'] = []


count = 1
for tid in titleid:
#     url = "https://comic.naver.com/webtoon/list.nhn?titleId="+ str(tid) +"&page=1"
#     response = requests.get(url)
#     dom = BeautifulSoup(response.content, "html.parser")
#     num_check = dom.select("#content > table.viewList > tr > td:nth-of-type(1) > a > img")
    print("webtoon id:",tid, " count: ", count)
    for num in range(1,201):
        
        url = "https://comic.naver.com/webtoon/list.nhn?titleId="+ str(tid) +"&page=" + str(num)
        response = requests.get(url)
        dom = BeautifulSoup(response.content, "html.parser")
        
        
        # 첫번째 링크가 광고인 경우를 제외시켜줘야 한다.
        links = dom.select("#content > table.viewList > tr > td:nth-of-type(1) > a > img")
        if len(links) > 10:
            links = links[1:]        
        # title
        for i in range(len(links)):
            dic2['title'].append(df[df.unique_id == tid].title.values[0])
            # image link
            dic2['image_link'].append(links[i].get('src'))
            # 회차(episode)
            dic2['episode'].append(links[i].get('title'))
        if len(links) < 10:
            break 
    count += 1

webtoon id: 651673  count:  1
webtoon id: 22052  count:  2
webtoon id: 498158  count:  3
webtoon id: 680911  count:  4
webtoon id: 15441  count:  5
webtoon id: 26316  count:  6
webtoon id: 409630  count:  7
webtoon id: 710747  count:  8
webtoon id: 679519  count:  9
webtoon id: 15938  count:  10
webtoon id: 651663  count:  11
webtoon id: 25517  count:  12
webtoon id: 639604  count:  13
webtoon id: 113121  count:  14
webtoon id: 316914  count:  15
webtoon id: 26101  count:  16
webtoon id: 70046  count:  17
webtoon id: 563782  count:  18
webtoon id: 20853  count:  19
webtoon id: 710751  count:  20
webtoon id: 26144  count:  21
webtoon id: 55169  count:  22
webtoon id: 52946  count:  23
webtoon id: 651664  count:  24
webtoon id: 632342  count:  25
webtoon id: 55143  count:  26
webtoon id: 609477  count:  27
webtoon id: 50422  count:  28
webtoon id: 642598  count:  29
webtoon id: 92106  count:  30
webtoon id: 710754  count:  31
webtoon id: 650304  count:  32
webtoon id: 316912  count:  33


webtoon id: 511447  count:  263
webtoon id: 25695  count:  264
webtoon id: 517773  count:  265
webtoon id: 26460  count:  266
webtoon id: 557676  count:  267
webtoon id: 702422  count:  268
webtoon id: 597481  count:  269
webtoon id: 459546  count:  270
webtoon id: 677737  count:  271
webtoon id: 670149  count:  272
webtoon id: 710761  count:  273
webtoon id: 552960  count:  274
webtoon id: 710760  count:  275
webtoon id: 26108  count:  276
webtoon id: 26107  count:  277
webtoon id: 142911  count:  278
webtoon id: 670150  count:  279
webtoon id: 651617  count:  280
webtoon id: 570506  count:  281
webtoon id: 25050  count:  282
webtoon id: 713975  count:  283
webtoon id: 698918  count:  284
webtoon id: 655748  count:  285
webtoon id: 699658  count:  286
webtoon id: 478262  count:  287
webtoon id: 337964  count:  288
webtoon id: 258205  count:  289
webtoon id: 714839  count:  290
webtoon id: 537990  count:  291
webtoon id: 551651  count:  292
webtoon id: 511446  count:  293
webtoon id: 6

webtoon id: 300957  count:  521
webtoon id: 58293  count:  522
webtoon id: 632705  count:  523
webtoon id: 452117  count:  524
webtoon id: 639080  count:  525
webtoon id: 656641  count:  526
webtoon id: 679568  count:  527
webtoon id: 629056  count:  528
webtoon id: 710752  count:  529
webtoon id: 579352  count:  530
webtoon id: 602930  count:  531
webtoon id: 697681  count:  532
webtoon id: 316913  count:  533
webtoon id: 703853  count:  534
webtoon id: 58219  count:  535
webtoon id: 462900  count:  536
webtoon id: 622646  count:  537
webtoon id: 710763  count:  538
webtoon id: 449854  count:  539
webtoon id: 604146  count:  540
webtoon id: 613933  count:  541
webtoon id: 703849  count:  542
webtoon id: 665170  count:  543
webtoon id: 710741  count:  544
webtoon id: 546623  count:  545
webtoon id: 703634  count:  546
webtoon id: 657948  count:  547
webtoon id: 703631  count:  548
webtoon id: 644112  count:  549
webtoon id: 709731  count:  550
webtoon id: 459545  count:  551
webtoon id

In [122]:
for k in dic2.keys():
    print(len(dic2[k]))

187691
187691
187691


In [123]:
df_episode = pd.DataFrame(dic2)
df_episode.tail()

,title,image_link,episode
187686,슈퍼트리오 시즌2,https://shared-comic.pstatic.net/thumb/webtoon...,Part 9. 의천검 탈취작전 1
187687,슈퍼트리오 시즌2,https://shared-comic.pstatic.net/thumb/webtoon...,Part 8. 쌍감자 체포 프로젝트 2
187688,슈퍼트리오 시즌2,https://shared-comic.pstatic.net/thumb/webtoon...,Part 8. 쌍감자 체포 프로젝트 1
187689,슈퍼트리오 시즌2,https://shared-comic.pstatic.net/thumb/webtoon...,Part 7. 일천불의 화가 2
187690,슈퍼트리오 시즌2,https://shared-comic.pstatic.net/thumb/webtoon...,Part 7. 일천불의 화가 1


In [128]:
df.to_csv("data_Juno/webtoon_naver_info_by_genres.csv", index=False)

In [126]:
df_episode.to_csv("data_Juno/episode_per_webtoon_naver.csv", index=False)

In [188]:
dic2['episode_count'] = []
count = 1
for tid in titleid:
#     url = "https://comic.naver.com/webtoon/list.nhn?titleId="+ str(tid) +"&page=1"
#     response = requests.get(url)
#     dom = BeautifulSoup(response.content, "html.parser")
#     num_check = dom.select("#content > table.viewList > tr > td:nth-of-type(1) > a > img")
    print("webtoon id:",tid, " count: ", count)
    for num in range(1,201):
        
        url = "https://comic.naver.com/webtoon/list.nhn?titleId="+ str(tid) +"&page=" + str(num)
        response = requests.get(url)
        dom = BeautifulSoup(response.content, "html.parser")
        
        
        # 첫번째 링크가 광고인 경우를 제외시켜줘야 한다.
        links = dom.select("#content > table.viewList > tr > td:nth-of-type(1) > a")
        if len(links) > 10:
            links = links[1:]        
        # episode count
        for i in range(len(links)):
            extract = links[i].get('onclick')
            extract = extract.split(',')[3][1:-1]
            dic2['episode_count'].append(extract)
        if len(links) < 10:
            break 
    count += 1

webtoon id: 651673  count:  1
webtoon id: 22052  count:  2
webtoon id: 498158  count:  3
webtoon id: 680911  count:  4
webtoon id: 15441  count:  5
webtoon id: 26316  count:  6
webtoon id: 409630  count:  7
webtoon id: 710747  count:  8
webtoon id: 679519  count:  9
webtoon id: 15938  count:  10
webtoon id: 651663  count:  11
webtoon id: 25517  count:  12
webtoon id: 639604  count:  13
webtoon id: 113121  count:  14
webtoon id: 316914  count:  15
webtoon id: 26101  count:  16
webtoon id: 70046  count:  17
webtoon id: 563782  count:  18
webtoon id: 20853  count:  19
webtoon id: 710751  count:  20
webtoon id: 26144  count:  21
webtoon id: 55169  count:  22
webtoon id: 52946  count:  23
webtoon id: 651664  count:  24
webtoon id: 632342  count:  25
webtoon id: 55143  count:  26
webtoon id: 609477  count:  27
webtoon id: 50422  count:  28
webtoon id: 642598  count:  29
webtoon id: 92106  count:  30
webtoon id: 710754  count:  31
webtoon id: 650304  count:  32
webtoon id: 316912  count:  33


webtoon id: 511447  count:  263
webtoon id: 25695  count:  264
webtoon id: 517773  count:  265
webtoon id: 26460  count:  266
webtoon id: 557676  count:  267
webtoon id: 702422  count:  268
webtoon id: 597481  count:  269
webtoon id: 459546  count:  270
webtoon id: 677737  count:  271
webtoon id: 670149  count:  272
webtoon id: 710761  count:  273
webtoon id: 552960  count:  274
webtoon id: 710760  count:  275
webtoon id: 26108  count:  276
webtoon id: 26107  count:  277
webtoon id: 142911  count:  278
webtoon id: 670150  count:  279
webtoon id: 651617  count:  280
webtoon id: 570506  count:  281
webtoon id: 25050  count:  282
webtoon id: 713975  count:  283
webtoon id: 698918  count:  284
webtoon id: 655748  count:  285
webtoon id: 699658  count:  286
webtoon id: 478262  count:  287
webtoon id: 337964  count:  288
webtoon id: 258205  count:  289
webtoon id: 714839  count:  290
webtoon id: 537990  count:  291
webtoon id: 551651  count:  292
webtoon id: 511446  count:  293
webtoon id: 6

webtoon id: 300957  count:  521
webtoon id: 58293  count:  522
webtoon id: 632705  count:  523
webtoon id: 452117  count:  524
webtoon id: 639080  count:  525
webtoon id: 656641  count:  526
webtoon id: 679568  count:  527
webtoon id: 629056  count:  528
webtoon id: 710752  count:  529
webtoon id: 579352  count:  530
webtoon id: 602930  count:  531
webtoon id: 697681  count:  532
webtoon id: 316913  count:  533
webtoon id: 703853  count:  534
webtoon id: 58219  count:  535
webtoon id: 462900  count:  536
webtoon id: 622646  count:  537
webtoon id: 710763  count:  538
webtoon id: 449854  count:  539
webtoon id: 604146  count:  540
webtoon id: 613933  count:  541
webtoon id: 703849  count:  542
webtoon id: 665170  count:  543
webtoon id: 710741  count:  544
webtoon id: 546623  count:  545
webtoon id: 703634  count:  546
webtoon id: 657948  count:  547
webtoon id: 703631  count:  548
webtoon id: 644112  count:  549
webtoon id: 709731  count:  550
webtoon id: 459545  count:  551
webtoon id

In [189]:
for k in dic2.keys():
    print(len(dic2[k]))

187691
187691
187691
250460


In [214]:
dic2['episode_count']

['4',
 '3',
 '2',
 '1',
 '10',
 '9',
 '8',
 '7',
 '6',
 '5',
 '4',
 '3',
 '2',
 '1',
 '10',
 '9',
 '8',
 '7',
 '6',
 '5',
 '4',
 '3',
 '2',
 '1',
 '10',
 '9',
 '8',
 '7',
 '6',
 '5',
 '4',
 '3',
 '2',
 '1',
 '10',
 '9',
 '8',
 '7',
 '6',
 '5',
 '4',
 '3',
 '2',
 '1',
 '10',
 '9',
 '8',
 '7',
 '6',
 '5',
 '4',
 '3',
 '2',
 '1',
 '10',
 '9',
 '8',
 '7',
 '6',
 '5',
 '4',
 '3',
 '2',
 '1',
 '16',
 '15',
 '14',
 '13',
 '12',
 '11',
 '10',
 '9',
 '8',
 '7',
 '',
 '6',
 '5',
 '4',
 '3',
 '2',
 '1',
 '12',
 '11',
 '10',
 '9',
 '8',
 '7',
 '6',
 '5',
 '4',
 '3',
 '',
 '2',
 '1',
 '',
 '6',
 '5',
 '4',
 '3',
 '2']

In [202]:
sample = {}
sample['cnt'] = []
for tid in titleid[-3:]:
#     url = "https://comic.naver.com/webtoon/list.nhn?titleId="+ str(tid) +"&page=1"
#     response = requests.get(url)
#     dom = BeautifulSoup(response.content, "html.parser")
#     num_check = dom.select("#content > table.viewList > tr > td:nth-of-type(1) > a > img")
    print("webtoon id:",tid, " count: ", count)
    for num in range(1,201):
        
        url = "https://comic.naver.com/webtoon/list.nhn?titleId="+ str(tid) +"&page=" + str(num)
        response = requests.get(url)
        dom = BeautifulSoup(response.content, "html.parser")
        
        
        # 첫번째 링크가 광고인 경우를 제외시켜줘야 한다.
        links = dom.select("#content > table.viewList > tr > td:nth-of-type(1) > a")
        if len(links) > 10:
            links = links[1:]        
        # episode count
        for i in range(len(links)):
            extract = links[i].get('onclick')
            extract = extract.split(',')[3][1:-1]
            sample['cnt'].append(extract)
        if len(links) < 10:
            break 
    count += 1


webtoon id: 336945  count:  746
webtoon id: 515883  count:  747
webtoon id: 347685  count:  748


['16',
 '15',
 '14',
 '13',
 '12',
 '11',
 '10',
 '9',
 '8',
 '7',
 '',
 '6',
 '5',
 '4',
 '3',
 '2',
 '1',
 '12',
 '11',
 '10',
 '9',
 '8',
 '7',
 '6',
 '5',
 '4',
 '3',
 '',
 '2',
 '1',
 '',
 '6',
 '5',
 '4',
 '3',
 '2']

In [215]:
def remove_values_from_list(the_list, val):
        while val in the_list:
            the_list.remove(val)

In [217]:
remove_values_from_list(dic2['episode_count'],'')
len(dic2['episode_count'])

243468

In [224]:
set(dic2['episode_count'])

'999'

In [226]:
dic2['episode_count'] = list(map(int, dic2['episode_count'])

In [237]:
dic2['episode_count'].index(1336)

62001

In [243]:
dic2['title']

TypeError: count() takes exactly one argument (0 given)

In [251]:
# 중복행 발견
df_copy = df_episode.drop_duplicates() # Dataframe의 중복행 제거
df_copy['image_link'].value_counts()

https://shared-comic.pstatic.net/thumb/webtoon/25455/477/thumbnail_202x120_d18127cf-afce-4f01-9705-6c8393227b8a.jpg     1
https://shared-comic.pstatic.net/thumb/webtoon/440447/15/inst_thumbnail_20120518174006.jpg                              1
https://shared-comic.pstatic.net/thumb/webtoon/22027/19/inst_thumbnail_22027_19.jpg                                     1
https://shared-comic.pstatic.net/thumb/webtoon/55169/46/inst_thumbnail_20090220180708.jpg                               1
https://shared-comic.pstatic.net/thumb/webtoon/602910/63/inst_thumbnail_20150213181318.jpg                              1
https://shared-comic.pstatic.net/thumb/webtoon/316908/7/inst_thumbnail_20110506115337.jpg                               1
https://shared-comic.pstatic.net/thumb/webtoon/642598/229/thumbnail_202x120_e9bf359f-bfe7-4aa9-b1ca-c24d2c6c14e1.jpg    1
https://shared-comic.pstatic.net/thumb/webtoon/597478/165/thumbnail_202x120_307bae39-3a86-4ed2-9c48-e9102f13bc76.jpg    1
https://shared-comic.pst

In [255]:
df_copy['image_link'].value_counts().max()

1

In [257]:
df_copy.shape

(54701, 3)

In [286]:
df_copy.to_csv("data_Juno/episode_per_webtoon_naver.csv", index=False)

In [275]:
# episode count 추출
sample = df_copy['image_link'].values[16]
sample

'https://shared-comic.pstatic.net/thumb/webtoon/651673/320/thumbnail_202x120_169987fd-b926-431b-a26f-d9b7819824f0.jpg'

In [276]:
print(sample.split('/'))
sample.split('/')[5], sample.split('/')[6]

['https:', '', 'shared-comic.pstatic.net', 'thumb', 'webtoon', '651673', '320', 'thumbnail_202x120_169987fd-b926-431b-a26f-d9b7819824f0.jpg']


('651673', '320')

In [277]:
df_copy['titleid'] = df_copy['image_link'].map(lambda x : x.split('/')[5] if len(x.split('/')) > 7 else 0)
df_copy['episode_cnt'] = df_copy['image_link'].map(lambda x : x.split('/')[6] if len(x.split('/')) > 7 else 0)
df_copy.tail()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,title,image_link,episode,titleid,episode_cnt
187686,슈퍼트리오 시즌2,https://shared-comic.pstatic.net/thumb/webtoon...,Part 9. 의천검 탈취작전 1,347685,6
187687,슈퍼트리오 시즌2,https://shared-comic.pstatic.net/thumb/webtoon...,Part 8. 쌍감자 체포 프로젝트 2,347685,5
187688,슈퍼트리오 시즌2,https://shared-comic.pstatic.net/thumb/webtoon...,Part 8. 쌍감자 체포 프로젝트 1,347685,4
187689,슈퍼트리오 시즌2,https://shared-comic.pstatic.net/thumb/webtoon...,Part 7. 일천불의 화가 2,347685,3
187690,슈퍼트리오 시즌2,https://shared-comic.pstatic.net/thumb/webtoon...,Part 7. 일천불의 화가 1,347685,2


In [283]:
idx = df_copy[df_copy['episode_cnt'] == 0].index
df_copy.drop(index=idx.values, inplace=True)
df_copy.loc[idx,:]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,title,image_link,episode,titleid,episode_cnt


In [285]:
df_copy.tail()

,title,image_link,episode,titleid,episode_cnt
187686,슈퍼트리오 시즌2,https://shared-comic.pstatic.net/thumb/webtoon...,Part 9. 의천검 탈취작전 1,347685,6
187687,슈퍼트리오 시즌2,https://shared-comic.pstatic.net/thumb/webtoon...,Part 8. 쌍감자 체포 프로젝트 2,347685,5
187688,슈퍼트리오 시즌2,https://shared-comic.pstatic.net/thumb/webtoon...,Part 8. 쌍감자 체포 프로젝트 1,347685,4
187689,슈퍼트리오 시즌2,https://shared-comic.pstatic.net/thumb/webtoon...,Part 7. 일천불의 화가 2,347685,3
187690,슈퍼트리오 시즌2,https://shared-comic.pstatic.net/thumb/webtoon...,Part 7. 일천불의 화가 1,347685,2
